In [1]:
import os
import sys
import tensorflow as tf
from tensorflow import logging
from tensorflow import gfile
import pprint
import cPickle
import numpy as np
import random
import math
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

output_dir = './outputs_varied_dataset2'
if not gfile.Exists(output_dir):
    gfile.MakeDirs(output_dir)

def get_default_params():
    return tf.contrib.training.HParams(
        z_dim = 100,
        init_conv_size = 4,
        g_channels = [128, 64, 32, 1],
        d_channels = [32, 64, 128, 256],
        batch_size = 128,
        learning_rate = 0.002,
        beta1 = 0.5,
        img_size = 32,
    )

hps = get_default_params()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
class MnistData(object):
    def __init__(self, mnist_train, z_dim, img_size):
        self._data = mnist_train
        self._example_num = len(self._data)
        self._z_data = np.random.standard_normal((self._example_num, z_dim))
        self._indicator = 0
        self._resize_mnist_img(img_size)
        self._random_shuffle()
    
    def _random_shuffle(self):
        p = np.random.permutation(self._example_num)
        self._z_data = self._z_data[p]
        self._data = self._data[p]
    
    def _resize_mnist_img(self, img_size):
        data = np.asarray(self._data * 255, np.uint8)
        data = data.reshape((self._example_num, 1, 28, 28))
        data = data.transpose((0, 2, 3, 1))
        new_data = []
        for i in range(self._example_num):
            img = data[i].reshape((28, 28))
            img = Image.fromarray(img)
            img = img.resize((img_size, img_size))
            img = np.asarray(img)
            img = img.reshape((img_size, img_size, 1))
            new_data.append(img)
        new_data = np.asarray(new_data, dtype=np.float32)
        new_data = new_data / 127.5 - 1
        self._data = new_data
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._example_num:
            self._random_shuffle()
            self._indicator = 0
            end_indicator = self._indicator + batch_size
        assert end_indicator < self._example_num
        
        batch_data = self._data[self._indicator: end_indicator]
        batch_z = self._z_data[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_z

mnist_data = MnistData(mnist.train.images, hps.z_dim, hps.img_size)
batch_data, batch_z = mnist_data.next_batch(5)
print batch_data.shape
print batch_z.shape

(5, 32, 32, 1)
(5, 100)


In [3]:
def conv2d_transpose(inputs, out_channel, name, training, with_bn_relu=True):
    with tf.variable_scope(name):
        conv2d_trans = tf.layers.conv2d_transpose(inputs, 
                                                  out_channel,
                                                  [5, 5],
                                                  strides=(2,2),
                                                  padding='SAME')
        if with_bn_relu:
            bn = tf.layers.batch_normalization(conv2d_trans, training=training)
            relu = tf.nn.relu(bn)
            return relu
        else:
            return conv2d_trans

def conv2d(inputs, out_channel, name, training):
    def leaky_relu(x, leak = 0.2, name = ''):
        return tf.maximum(x, x * leak, name = name)
    with tf.variable_scope(name):
        conv2d_output = tf.layers.conv2d(inputs,
                                         out_channel,
                                         [5, 5],
                                         strides = (2, 2),
                                         padding = 'SAME')
        bn = tf.layers.batch_normalization(conv2d_output,
                                           training = training)
        return leaky_relu(bn, name = 'outputs')


class Generator(object):
    def __init__(self, channels, init_conv_size):
        assert len(channels) > 1
        self._channels = channels
        self._init_conv_size = init_conv_size
        self._reuse = False
    
    def __call__(self, inputs, training):
        inputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('generator', reuse=self._reuse):
            with tf.variable_scope('inputs'):
                fc = tf.layers.dense(
                    inputs, 
                    self._channels[0] * self._init_conv_size * self._init_conv_size)
                conv0 = tf.reshape(fc, [-1, self._init_conv_size, self._init_conv_size, self._channels[0]])
                bn0 = tf.layers.batch_normalization(conv0, training=training)
                relu0 = tf.nn.relu(bn0)
            
            deconv_inputs = relu0
            # deconvolutions * 4
            for i in range(1, len(self._channels)):
                with_bn_relu = (i != len(self._channels) - 1)
                deconv_inputs = conv2d_transpose(deconv_inputs,
                                                 self._channels[i],
                                                 'deconv-%d' % i,
                                                 training,
                                                 with_bn_relu)
            img_inputs = deconv_inputs
            with tf.variable_scope('generate_imgs'):
                # imgs value scope: [-1, 1]
                imgs = tf.tanh(img_inputs, name='imgaes')
        self._reuse=True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
        return imgs
    
        
class Discriminator(object):
    def __init__(self, channels):
        self._channels = channels
        self._reuse = False
    
    def __call__(self, inputs, training):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        
        conv_inputs = inputs
        with tf.variable_scope('discriminator', reuse = self._reuse):
            for i in range(len(self._channels)):
                conv_inputs = conv2d(conv_inputs,
                                     self._channels[i],
                                     'deconv-%d' % i,
                                     training)
            fc_inputs = conv_inputs
            with tf.variable_scope('fc'):
                flatten = tf.layers.flatten(fc_inputs)
                logits = tf.layers.dense(flatten, 2, name="logits")
        self._reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
        return logits


class DCGAN(object):
    def __init__(self, hps):
        g_channels = hps.g_channels
        d_channels = hps.d_channels
        self._batch_size = hps.batch_size
        self._init_conv_size = hps.init_conv_size
        self._batch_size = hps.batch_size
        self._z_dim = hps.z_dim
        self._img_size = hps.img_size
        
        self._generator = Generator(g_channels, self._init_conv_size)
        self._discriminator = Discriminator(d_channels)
    
    def build(self):
        self._z_placholder = tf.placeholder(tf.float32, (self._batch_size, self._z_dim))
        self._img_placeholder = tf.placeholder(tf.float32, 
                                               (self._batch_size, self._img_size, self._img_size, 1))
        
        generated_imgs = self._generator(self._z_placholder, training = True)
        fake_img_logits = self._discriminator(generated_imgs, training = True)
        real_img_logits = self._discriminator(self._img_placeholder, training = True)
        
        loss_on_fake_to_real = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size], dtype = tf.int64),
                logits = fake_img_logits))
        loss_on_fake_to_fake = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.zeros([self._batch_size], dtype = tf.int64),
                logits = fake_img_logits))
        loss_on_real_to_real = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size], dtype = tf.int64),
                logits = real_img_logits))
        
        tf.add_to_collection('g_losses', loss_on_fake_to_real)
        tf.add_to_collection('d_losses', loss_on_fake_to_fake)
        tf.add_to_collection('d_losses', loss_on_real_to_real)
        
        
        loss = {
            'g': tf.add_n(tf.get_collection('g_losses'), name = 'total_g_loss'),
            'd': tf.add_n(tf.get_collection('d_losses'), name = 'total_d_loss')
        }
        
        return (self._z_placholder, self._img_placeholder, generated_imgs, loss)
    
    def build_train(self, losses, learning_rate, beta1):
        g_opt = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1)
        d_opt = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1)
        g_opt_op = g_opt.minimize(losses['g'], var_list = self._generator.variables)
        d_opt_op = d_opt.minimize(losses['d'], var_list = self._discriminator.variables)
        with tf.control_dependencies([g_opt_op, d_opt_op]):
            return tf.no_op(name = 'train')


dcgan = DCGAN(hps)
z_placeholder, img_placeholder, generated_imgs, losses = dcgan.build()
train_op = dcgan.build_train(losses, hps.learning_rate, hps.beta1)


In [4]:
def combine_and_show_imgs(batch_imgs, img_size, rows=8, cols=16):
    # batch_imgs: [batch_size, img_size, img_size, 1]
    result_big_img = []
    for i in range(rows):
        row_imgs = []
        for j in range(cols):
            img = batch_imgs[cols * i + j]
            img = img.reshape((img_size, img_size))
            img = (img + 1) * 127.5
            row_imgs.append(img)
        row_imgs = np.hstack(row_imgs)
        result_big_img.append(row_imgs)
    result_big_img = np.vstack(result_big_img)
    result_big_img = np.asarray(result_big_img, np.uint8)
    result_big_img = Image.fromarray(result_big_img)
    return result_big_img

            
    
    
init_op = tf.global_variables_initializer()
train_steps = 10000

with tf.Session() as sess:
    sess.run(init_op)
    for step in range(train_steps):
        batch_img, batch_z = mnist_data.next_batch(hps.batch_size)
        
        fetches = [train_op, losses['g'], losses['d']]
        should_sample = (step + 1) % 50 == 0
        if should_sample:
            fetches += [generated_imgs]
        out_values = sess.run(fetches,
                              feed_dict = {
                                  z_placeholder: batch_z,
                                  img_placeholder: batch_img
                              })
        _, g_loss_val, d_loss_val = out_values[0:3]
        logging.info('step: %d, g_loss: %4.3f, d_loss: %4.3f' % (step, g_loss_val, d_loss_val))
        if should_sample:
            gen_imgs_val = out_values[3]
            
            gen_img_path = os.path.join(output_dir, '%05d-gen.jpg' % (step + 1))
            gt_img_path = os.path.join(output_dir, '%05d-gt.jpg' % (step + 1))
            
            gen_img = combine_and_show_imgs(gen_imgs_val, hps.img_size)
            gt_img = combine_and_show_imgs(batch_img, hps.img_size)
            
            gen_img.save(gen_img_path)
            gt_img.save(gt_img_path)
        

INFO:tensorflow:step: 0, g_loss: 0.712, d_loss: 2.008
INFO:tensorflow:step: 1, g_loss: 10.005, d_loss: 0.001
INFO:tensorflow:step: 2, g_loss: 0.000, d_loss: 14.478
INFO:tensorflow:step: 3, g_loss: 25.793, d_loss: 0.479
INFO:tensorflow:step: 4, g_loss: 25.101, d_loss: 0.000
INFO:tensorflow:step: 5, g_loss: 24.325, d_loss: 0.000
INFO:tensorflow:step: 6, g_loss: 23.230, d_loss: 0.000
INFO:tensorflow:step: 7, g_loss: 19.744, d_loss: 0.000
INFO:tensorflow:step: 8, g_loss: 2.124, d_loss: 0.608
INFO:tensorflow:step: 9, g_loss: 21.160, d_loss: 0.046
INFO:tensorflow:step: 10, g_loss: 20.155, d_loss: 0.217
INFO:tensorflow:step: 11, g_loss: 0.333, d_loss: 2.200
INFO:tensorflow:step: 12, g_loss: 21.101, d_loss: 13.246
INFO:tensorflow:step: 13, g_loss: 15.638, d_loss: 1.459
INFO:tensorflow:step: 14, g_loss: 8.826, d_loss: 0.000
INFO:tensorflow:step: 15, g_loss: 1.676, d_loss: 0.234
INFO:tensorflow:step: 16, g_loss: 0.190, d_loss: 1.994
INFO:tensorflow:step: 17, g_loss: 11.212, d_loss: 0.000
INFO:te

INFO:tensorflow:step: 148, g_loss: 5.077, d_loss: 0.094
INFO:tensorflow:step: 149, g_loss: 1.943, d_loss: 0.551
INFO:tensorflow:step: 150, g_loss: 4.909, d_loss: 0.519
INFO:tensorflow:step: 151, g_loss: 0.829, d_loss: 1.142
INFO:tensorflow:step: 152, g_loss: 7.447, d_loss: 1.905
INFO:tensorflow:step: 153, g_loss: 2.327, d_loss: 0.170
INFO:tensorflow:step: 154, g_loss: 0.134, d_loss: 2.275
INFO:tensorflow:step: 155, g_loss: 6.839, d_loss: 0.757
INFO:tensorflow:step: 156, g_loss: 6.989, d_loss: 0.454
INFO:tensorflow:step: 157, g_loss: 3.932, d_loss: 0.084
INFO:tensorflow:step: 158, g_loss: 0.906, d_loss: 0.654
INFO:tensorflow:step: 159, g_loss: 4.804, d_loss: 0.216
INFO:tensorflow:step: 160, g_loss: 5.094, d_loss: 0.294
INFO:tensorflow:step: 161, g_loss: 2.692, d_loss: 0.193
INFO:tensorflow:step: 162, g_loss: 1.263, d_loss: 0.510
INFO:tensorflow:step: 163, g_loss: 2.920, d_loss: 0.331
INFO:tensorflow:step: 164, g_loss: 0.705, d_loss: 1.076
INFO:tensorflow:step: 165, g_loss: 4.651, d_loss

INFO:tensorflow:step: 295, g_loss: 3.256, d_loss: 0.263
INFO:tensorflow:step: 296, g_loss: 3.121, d_loss: 0.741
INFO:tensorflow:step: 297, g_loss: 0.429, d_loss: 1.351
INFO:tensorflow:step: 298, g_loss: 3.727, d_loss: 1.311
INFO:tensorflow:step: 299, g_loss: 0.541, d_loss: 1.292
INFO:tensorflow:step: 300, g_loss: 1.984, d_loss: 0.683
INFO:tensorflow:step: 301, g_loss: 1.867, d_loss: 0.535
INFO:tensorflow:step: 302, g_loss: 1.351, d_loss: 0.530
INFO:tensorflow:step: 303, g_loss: 2.389, d_loss: 0.307
INFO:tensorflow:step: 304, g_loss: 1.853, d_loss: 0.438
INFO:tensorflow:step: 305, g_loss: 2.195, d_loss: 0.394
INFO:tensorflow:step: 306, g_loss: 1.503, d_loss: 0.491
INFO:tensorflow:step: 307, g_loss: 2.756, d_loss: 0.614
INFO:tensorflow:step: 308, g_loss: 0.347, d_loss: 1.394
INFO:tensorflow:step: 309, g_loss: 7.738, d_loss: 2.293
INFO:tensorflow:step: 310, g_loss: 2.376, d_loss: 0.237
INFO:tensorflow:step: 311, g_loss: 0.709, d_loss: 0.859
INFO:tensorflow:step: 312, g_loss: 3.380, d_loss

INFO:tensorflow:step: 442, g_loss: 0.996, d_loss: 0.609
INFO:tensorflow:step: 443, g_loss: 3.097, d_loss: 0.346
INFO:tensorflow:step: 444, g_loss: 2.926, d_loss: 0.216
INFO:tensorflow:step: 445, g_loss: 1.052, d_loss: 0.745
INFO:tensorflow:step: 446, g_loss: 3.174, d_loss: 0.776
INFO:tensorflow:step: 447, g_loss: 0.603, d_loss: 1.116
INFO:tensorflow:step: 448, g_loss: 4.411, d_loss: 1.231
INFO:tensorflow:step: 449, g_loss: 0.506, d_loss: 1.282
INFO:tensorflow:step: 450, g_loss: 4.708, d_loss: 0.437
INFO:tensorflow:step: 451, g_loss: 3.416, d_loss: 0.343
INFO:tensorflow:step: 452, g_loss: 0.650, d_loss: 1.022
INFO:tensorflow:step: 453, g_loss: 4.104, d_loss: 1.159
INFO:tensorflow:step: 454, g_loss: 0.527, d_loss: 1.234
INFO:tensorflow:step: 455, g_loss: 3.877, d_loss: 0.768
INFO:tensorflow:step: 456, g_loss: 1.843, d_loss: 0.507
INFO:tensorflow:step: 457, g_loss: 0.728, d_loss: 0.882
INFO:tensorflow:step: 458, g_loss: 3.124, d_loss: 0.636
INFO:tensorflow:step: 459, g_loss: 2.061, d_loss

INFO:tensorflow:step: 589, g_loss: 4.824, d_loss: 0.265
INFO:tensorflow:step: 590, g_loss: 3.836, d_loss: 0.261
INFO:tensorflow:step: 591, g_loss: 1.627, d_loss: 0.358
INFO:tensorflow:step: 592, g_loss: 1.555, d_loss: 0.543
INFO:tensorflow:step: 593, g_loss: 2.275, d_loss: 0.580
INFO:tensorflow:step: 594, g_loss: 1.289, d_loss: 0.578
INFO:tensorflow:step: 595, g_loss: 1.701, d_loss: 0.597
INFO:tensorflow:step: 596, g_loss: 2.478, d_loss: 0.569
INFO:tensorflow:step: 597, g_loss: 1.116, d_loss: 0.558
INFO:tensorflow:step: 598, g_loss: 4.350, d_loss: 0.747
INFO:tensorflow:step: 599, g_loss: 0.864, d_loss: 0.763
INFO:tensorflow:step: 600, g_loss: 3.963, d_loss: 0.293
INFO:tensorflow:step: 601, g_loss: 3.116, d_loss: 0.140
INFO:tensorflow:step: 602, g_loss: 1.681, d_loss: 0.494
INFO:tensorflow:step: 603, g_loss: 1.967, d_loss: 0.488
INFO:tensorflow:step: 604, g_loss: 2.417, d_loss: 0.502
INFO:tensorflow:step: 605, g_loss: 1.562, d_loss: 0.453
INFO:tensorflow:step: 606, g_loss: 4.411, d_loss

...

INFO:tensorflow:step: 9381, g_loss: 3.275, d_loss: 0.271
INFO:tensorflow:step: 9382, g_loss: 3.158, d_loss: 0.113
INFO:tensorflow:step: 9383, g_loss: 3.243, d_loss: 0.150
INFO:tensorflow:step: 9384, g_loss: 4.293, d_loss: 0.045
INFO:tensorflow:step: 9385, g_loss: 3.817, d_loss: 0.092
INFO:tensorflow:step: 9386, g_loss: 3.135, d_loss: 0.220
INFO:tensorflow:step: 9387, g_loss: 3.326, d_loss: 0.200
INFO:tensorflow:step: 9388, g_loss: 3.031, d_loss: 0.105
INFO:tensorflow:step: 9389, g_loss: 4.446, d_loss: 0.057
INFO:tensorflow:step: 9390, g_loss: 4.463, d_loss: 0.125
INFO:tensorflow:step: 9391, g_loss: 2.150, d_loss: 0.243
INFO:tensorflow:step: 9392, g_loss: 6.116, d_loss: 0.129
INFO:tensorflow:step: 9393, g_loss: 4.017, d_loss: 0.064
INFO:tensorflow:step: 9394, g_loss: 4.561, d_loss: 0.106
INFO:tensorflow:step: 9395, g_loss: 4.303, d_loss: 0.064
INFO:tensorflow:step: 9396, g_loss: 4.217, d_loss: 0.030
INFO:tensorflow:step: 9397, g_loss: 3.368, d_loss: 0.067
INFO:tensorflow:step: 9398, g_l

INFO:tensorflow:step: 9525, g_loss: 3.133, d_loss: 0.120
INFO:tensorflow:step: 9526, g_loss: 2.624, d_loss: 0.157
INFO:tensorflow:step: 9527, g_loss: 4.965, d_loss: 0.039
INFO:tensorflow:step: 9528, g_loss: 4.030, d_loss: 0.157
INFO:tensorflow:step: 9529, g_loss: 4.681, d_loss: 0.043
INFO:tensorflow:step: 9530, g_loss: 5.632, d_loss: 0.018
INFO:tensorflow:step: 9531, g_loss: 1.945, d_loss: 0.306
INFO:tensorflow:step: 9532, g_loss: 6.704, d_loss: 0.347
INFO:tensorflow:step: 9533, g_loss: 3.469, d_loss: 0.101
INFO:tensorflow:step: 9534, g_loss: 3.393, d_loss: 0.098
INFO:tensorflow:step: 9535, g_loss: 3.611, d_loss: 0.068
INFO:tensorflow:step: 9536, g_loss: 4.116, d_loss: 0.055
INFO:tensorflow:step: 9537, g_loss: 5.765, d_loss: 0.033
INFO:tensorflow:step: 9538, g_loss: 4.213, d_loss: 0.098
INFO:tensorflow:step: 9539, g_loss: 3.973, d_loss: 0.042
INFO:tensorflow:step: 9540, g_loss: 4.184, d_loss: 0.078
INFO:tensorflow:step: 9541, g_loss: 4.306, d_loss: 0.068
INFO:tensorflow:step: 9542, g_l

INFO:tensorflow:step: 9669, g_loss: 1.302, d_loss: 0.610
INFO:tensorflow:step: 9670, g_loss: 4.428, d_loss: 0.239
INFO:tensorflow:step: 9671, g_loss: 4.075, d_loss: 0.165
INFO:tensorflow:step: 9672, g_loss: 2.405, d_loss: 0.201
INFO:tensorflow:step: 9673, g_loss: 4.242, d_loss: 0.119
INFO:tensorflow:step: 9674, g_loss: 2.852, d_loss: 0.314
INFO:tensorflow:step: 9675, g_loss: 2.764, d_loss: 0.144
INFO:tensorflow:step: 9676, g_loss: 2.558, d_loss: 0.191
INFO:tensorflow:step: 9677, g_loss: 5.647, d_loss: 0.186
INFO:tensorflow:step: 9678, g_loss: 4.534, d_loss: 0.024
INFO:tensorflow:step: 9679, g_loss: 2.108, d_loss: 0.228
INFO:tensorflow:step: 9680, g_loss: 5.568, d_loss: 0.227
INFO:tensorflow:step: 9681, g_loss: 2.153, d_loss: 0.290
INFO:tensorflow:step: 9682, g_loss: 7.874, d_loss: 0.732
INFO:tensorflow:step: 9683, g_loss: 0.327, d_loss: 1.850
INFO:tensorflow:step: 9684, g_loss: 7.818, d_loss: 1.494
INFO:tensorflow:step: 9685, g_loss: 3.113, d_loss: 0.122
INFO:tensorflow:step: 9686, g_l

INFO:tensorflow:step: 9813, g_loss: 7.066, d_loss: 0.022
INFO:tensorflow:step: 9814, g_loss: 4.729, d_loss: 0.056
INFO:tensorflow:step: 9815, g_loss: 3.990, d_loss: 0.043
INFO:tensorflow:step: 9816, g_loss: 3.338, d_loss: 0.086
INFO:tensorflow:step: 9817, g_loss: 6.729, d_loss: 0.072
INFO:tensorflow:step: 9818, g_loss: 4.547, d_loss: 0.039
INFO:tensorflow:step: 9819, g_loss: 4.970, d_loss: 0.058
INFO:tensorflow:step: 9820, g_loss: 3.124, d_loss: 0.107
INFO:tensorflow:step: 9821, g_loss: 4.186, d_loss: 0.036
INFO:tensorflow:step: 9822, g_loss: 6.540, d_loss: 0.024
INFO:tensorflow:step: 9823, g_loss: 5.240, d_loss: 0.047
INFO:tensorflow:step: 9824, g_loss: 5.064, d_loss: 0.023
INFO:tensorflow:step: 9825, g_loss: 4.321, d_loss: 0.036
INFO:tensorflow:step: 9826, g_loss: 2.997, d_loss: 0.106
INFO:tensorflow:step: 9827, g_loss: 6.439, d_loss: 0.090
INFO:tensorflow:step: 9828, g_loss: 5.552, d_loss: 0.050
INFO:tensorflow:step: 9829, g_loss: 2.812, d_loss: 0.123
INFO:tensorflow:step: 9830, g_l

INFO:tensorflow:step: 9957, g_loss: 4.787, d_loss: 0.890
INFO:tensorflow:step: 9958, g_loss: 2.403, d_loss: 0.201
INFO:tensorflow:step: 9959, g_loss: 0.754, d_loss: 0.962
INFO:tensorflow:step: 9960, g_loss: 5.545, d_loss: 1.075
INFO:tensorflow:step: 9961, g_loss: 2.387, d_loss: 0.183
INFO:tensorflow:step: 9962, g_loss: 2.382, d_loss: 0.271
INFO:tensorflow:step: 9963, g_loss: 4.031, d_loss: 0.173
INFO:tensorflow:step: 9964, g_loss: 3.177, d_loss: 0.120
INFO:tensorflow:step: 9965, g_loss: 4.585, d_loss: 0.067
INFO:tensorflow:step: 9966, g_loss: 4.385, d_loss: 0.469
INFO:tensorflow:step: 9967, g_loss: 1.235, d_loss: 0.549
INFO:tensorflow:step: 9968, g_loss: 1.889, d_loss: 0.429
INFO:tensorflow:step: 9969, g_loss: 6.085, d_loss: 0.756
INFO:tensorflow:step: 9970, g_loss: 1.359, d_loss: 0.580
INFO:tensorflow:step: 9971, g_loss: 4.337, d_loss: 0.315
INFO:tensorflow:step: 9972, g_loss: 2.719, d_loss: 0.170
INFO:tensorflow:step: 9973, g_loss: 3.978, d_loss: 0.104
INFO:tensorflow:step: 9974, g_l